In [ ]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
# edu=enduser/table structure
edutable=""" CREATE TABLE user (
            Userid VARCHAR(255) NOT NULL PRIMARY KEY,
            First_Name CHAR(25) NOT NULL,
            Last_Name CHAR(25),
            Status INTEGER(25),
            Email VARCHAR(255),
            Password VARCHAR(255) NOT NULL,
            Fine INT
            ); """

#creates the table
cursor.execute(edutable)
cursor.close()
conn.close()

In [1]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE user")
conn.close()

In [20]:
import requests

response = requests.get("https://openlibrary.org/search.json?q=*&sort=rating&limit=100")
data = response.json()

In [44]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE book")
conn.close()

In [45]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
bktable=""" CREATE TABLE book (
            Bookid VARCHAR(255) NOT NULL PRIMARY KEY,
            Title CHAR(25) NOT NULL,
            Author CHAR(25),
            Status INTEGER(25),
            Description TEXT,
            Date TEXT
            ); """

#creates the table
cursor.execute(bktable)
conn.close()

In [ ]:
import requests
import sqlite3
from dotenv import load_dotenv
import os
APIKEY = os.getenv("API_KEY")

conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

j=0
for  i in data["docs"]:
    params = {
        "q": i["title"], 
        "key":APIKEY,
    }
    response = requests.get("https://www.googleapis.com/books/v1/volumes",params=params)
    response.raise_for_status()
    data2 = response.json()
    data3=data2['items'][0]['volumeInfo']
    query = f"""INSERT INTO book 
                (Bookid, Title, Author, Status, Description, Date) 
                VALUES (?, ?, ?, ?, ?, ?)"""
    values = (
    f'B{j}', 
    i["title"], 
    data3['authors'][0], 
    0, 
    data3['description'],
    data3['publishedDate'],
    )
    cursor.execute(query, values)
    conn.commit()
    j=j+1
conn.close()
    

KeyError: 'description'

In [ ]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
btable = """
CREATE TABLE IF NOT EXISTS borrow (
    BorrowID INTEGER PRIMARY KEY AUTOINCREMENT,
    UserID VARCHAR(255) NOT NULL,
    BookID VARCHAR(255) NOT NULL,
    BorrowDate TEXT NOT NULL DEFAULT CURRENT_DATE,
    DueDate TEXT NOT NULL,
    FOREIGN KEY (UserID) REFERENCES user(UserID) ON DELETE CASCADE,
    FOREIGN KEY (BookID) REFERENCES book(BookID) ON DELETE CASCADE
);
"""
cursor.execute(btable)
conn.close()

In [11]:
import sqlite3
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE borrowing")
conn.close()

In [19]:
from datetime import date
today = date.today()
print(today)
mon=today.month
pas=today.day
pas=pas+7
if(mon%2==0):
    if(pas>31):
        pas=pas-31
    else:
        pas=pas-30
pas

2025-05-18


25